In [50]:
import pandas as pd
mangaworld_df = pd.read_csv("mangaworld_normalized.csv")

mangaworld_df.head()

title  \
0  +99 Reinforced Wooden Stick   
1                       .traeH   
2                009 Re:Cyborg   
3  1-nen C-gumi Yoshida Kakari   
4                   1/2 Prince   

                                          alt_titles          author  \
0  ['+99 Wooden Stick', '+99 ท่อนไม้พร้อมบวก', '9...         Hongsil   
1                                           ['.قلب']    HANATSU Yaya   
2  ['009 Re: Cyborg', '009 Re:사이보그', '009 Ре:Кибо...             NaN   
3  ['1 Nen C Gumi Yoshida Kakari', '1年C組吉田係', 'Fi...  SAKURAI Shiori   
4  ['1/2 Hoàng Tử', '1/2 Princas', '1/2 Wangzi', ...           Yu Wo   

            artist     type     status  year    views  \
0           Jipery   Manhwa    dropped  2022  1385162   
1     HANATSU Yaya  Oneshot  completed  2008    62678   
2              NaN    Manga  completed  2012    44944   
3   SAKURAI Shiori  Oneshot  completed  2011    46507   
4  Choi Hong Chong   Manhua  completed  2006   386591   

              title_normalized  
0   99 reinforced wooden stick  
1                        traeh  
2                 009 recyborg  
3  1-nen c-gumi yoshida kakari  
4                    12 prince

In [51]:
mangaworld_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2906 entries, 0 to 2905
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             2906 non-null   object
 1   alt_titles        2855 non-null   object
 2   author            2609 non-null   object
 3   artist            2609 non-null   object
 4   type              2906 non-null   object
 5   status            2906 non-null   object
 6   year              2906 non-null   int64 
 7   views             2906 non-null   int64 
 8   title_normalized  2906 non-null   object
dtypes: int64(2), object(7)
memory usage: 204.5+ KB


In [52]:

import re
mangaworld_df["alt_titles"] = mangaworld_df["alt_titles"].str.lower()
mangaworld_df["alt_titles"] = mangaworld_df["alt_titles"].str.replace(r"[^\w\s\[\],’'”“‘]", "", regex=True)
mangaworld_df["alt_titles"] = mangaworld_df["alt_titles"].str.replace(r"\s+", " ", regex=True).str.strip()

mangaworld_df["alt_titles"].head(10)


0    ['99 wooden stick', '99 ทอนไมพรอมบวก', '99 har...
1                                              ['قلب']
2      ['009 re cyborg', '009 re사이보그', '009 рекиборг']
3    ['1 nen c gumi yoshida kakari', '1年c組吉田係', 'fi...
4    ['12 hoàng tử', '12 princas', '12 wangzi', '12...
5    ['13 romantika', '13ロマンチカ', 'centimeter na fut...
6    ['13 san bun no ichi', '13 さんぶんのいち', '13 auf e...
7              ['10년째 차이는 소꿉친구', '振られ続けて10年目相手は幼馴染です']
8    ['100 days until i collapse from hunger and be...
9    ['100 morango brazilian portuguese', '100 stra...
Name: alt_titles, dtype: object

In [53]:
from lingua import LanguageDetectorBuilder
detector = LanguageDetectorBuilder.from_all_languages().build()

def split_and_detect_languages(alt_titles):
    if pd.isna(alt_titles):
        return {}
    
    titles = alt_titles.strip("[]").split(",")
    titles = [title.strip() for title in titles if title.strip()]
    
    lang_dict = {}
    for title in titles:
        try:
            language = detector.detect_language_of(title)
            lang_name = language.name.lower()
            if lang_name not in lang_dict:
                lang_dict[lang_name] = []
            lang_dict[lang_name].append(title)
        except:
            continue
            
    return lang_dict

mangaworld_df["alt_titles_by_language"] = mangaworld_df["alt_titles"].apply(split_and_detect_languages)
mangaworld_df[["alt_titles", "alt_titles_by_language"]].head(10)


alt_titles  \
0  ['99 wooden stick', '99 ทอนไมพรอมบวก', '99 har...   
1                                            ['قلب']   
2    ['009 re cyborg', '009 re사이보그', '009 рекиборг']   
3  ['1 nen c gumi yoshida kakari', '1年c組吉田係', 'fi...   
4  ['12 hoàng tử', '12 princas', '12 wangzi', '12...   
5  ['13 romantika', '13ロマンチカ', 'centimeter na fut...   
6  ['13 san bun no ichi', '13 さんぶんのいち', '13 auf e...   
7            ['10년째 차이는 소꿉친구', '振られ続けて10年目相手は幼馴染です']   
8  ['100 days until i collapse from hunger and be...   
9  ['100 morango brazilian portuguese', '100 stra...   

                              alt_titles_by_language  
0  {'slovak': [''99 wooden stick''], 'thai': [''9...  
1                              {'arabic': [''قلب'']}  
2  {'danish': [''009 re cyborg''], 'korean': [''0...  
3  {'swahili': [''1 nen c gumi yoshida kakari'', ...  
4  {'vietnamese': [''12 hoàng tử''], 'italian': [...  
5  {'esperanto': [''13 romantika''], 'japanese': ...  
6  {'shona': [''13 san bun no ichi''], 'japanese'...  
7  {'korean': [''10년째 차이는 소꿉친구''], 'japanese': ['...  
8  {'english': [''100 days until i collapse from ...  
9  {'portuguese': [''100 morango brazilian portug...

In [54]:
def split_and_detect_languages(alt_titles):
    if pd.isna(alt_titles):
        return {}
    
    titles = alt_titles.strip("[]").split(",")
    titles = [title.strip() for title in titles if title.strip()]
    lang_dict = {}
    
    for title in titles:
        try:
            language = detector.detect_language_of(title)
            lang_name = language.name.lower()
            if lang_name not in lang_dict:
                lang_dict[lang_name] = []
            lang_dict[lang_name].append(title)
        except:
            continue
            
    return lang_dict

mangaworld_df["alt_titles_by_language"] = mangaworld_df["alt_titles"].apply(split_and_detect_languages)

alt_titles_by_language_df = pd.json_normalize(mangaworld_df["alt_titles_by_language"])

mangaworld_df = pd.concat([mangaworld_df, alt_titles_by_language_df], axis=1)

mangaworld_df.head()


title  \
0  +99 Reinforced Wooden Stick   
1                       .traeH   
2                009 Re:Cyborg   
3  1-nen C-gumi Yoshida Kakari   
4                   1/2 Prince   

                                          alt_titles          author  \
0  ['99 wooden stick', '99 ทอนไมพรอมบวก', '99 har...         Hongsil   
1                                            ['قلب']    HANATSU Yaya   
2    ['009 re cyborg', '009 re사이보그', '009 рекиборг']             NaN   
3  ['1 nen c gumi yoshida kakari', '1年c組吉田係', 'fi...  SAKURAI Shiori   
4  ['12 hoàng tử', '12 princas', '12 wangzi', '12...           Yu Wo   

            artist     type     status  year    views  \
0           Jipery   Manhwa    dropped  2022  1385162   
1     HANATSU Yaya  Oneshot  completed  2008    62678   
2              NaN    Manga  completed  2012    44944   
3   SAKURAI Shiori  Oneshot  completed  2011    46507   
4  Choi Hong Chong   Manhua  completed  2006   386591   

              title_normalized  \
0   99 reinforced wooden stick   
1                        traeh   
2                 009 recyborg   
3  1-nen c-gumi yoshida kakari   
4                    12 prince   

                              alt_titles_by_language  ... tamil hindi slovene  \
0  {'slovak': [''99 wooden stick''], 'thai': [''9...  ...   NaN   NaN     NaN   
1                              {'arabic': [''قلب'']}  ...   NaN   NaN     NaN   
2  {'danish': [''009 re cyborg''], 'korean': [''0...  ...   NaN   NaN     NaN   
3  {'swahili': [''1 nen c gumi yoshida kakari'', ...  ...   NaN   NaN     NaN   
4  {'vietnamese': [''12 hoàng tử''], 'italian': [...  ...   NaN   NaN     NaN   

  albanian czech kazakh icelandic latvian azerbaijani urdu  
0      NaN   NaN    NaN       NaN     NaN         NaN  NaN  
1      NaN   NaN    NaN       NaN     NaN         NaN  NaN  
2      NaN   NaN    NaN       NaN     NaN         NaN  NaN  
3      NaN   NaN    NaN       NaN     NaN         NaN  NaN  
4      NaN   NaN    NaN       NaN     NaN         NaN  NaN  

[5 rows x 84 columns]

In [ ]:
for col in alt_titles_by_language_df.columns:
    alt_titles_by_language_df[col] = alt_titles_by_language_df[col].apply(lambda x: list(dict.fromkeys(x)) if isinstance(x, list) else x)

mangaworld_df.update(alt_titles_by_language_df)

alt_titles_by_language_df.info()


ValueError: Array conditional must be same shape as self

In [ ]:
from difflib import SequenceMatcher

SIMILARITY_THRESHOLD = 0.85

def remove_duplicates_and_similar(titles):
    if not isinstance(titles, list):
        return titles
    
    unique_titles = list(dict.fromkeys(titles))
    
    final_titles = []
    for title in unique_titles:
        if not any(SequenceMatcher(None, title, t).ratio() > SIMILARITY_THRESHOLD for t in final_titles):
            final_titles.append(title)
    
    return final_titles

for col in alt_titles_by_language_df.columns:
    alt_titles_by_language_df[col] = alt_titles_by_language_df[col].apply(remove_duplicates_and_similar)

mangaworld_df.update(alt_titles_by_language_df)

alt_titles_by_language_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2906 entries, 0 to 2905
Data columns (total 74 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   slovak       19 non-null     object
 1   thai         470 non-null    object
 2   english      1518 non-null   object
 3   chinese      1653 non-null   object
 4   korean       961 non-null    object
 5   arabic       265 non-null    object
 6   danish       34 non-null     object
 7   mongolian    44 non-null     object
 8   swahili      312 non-null    object
 9   russian      528 non-null    object
 10  vietnamese   260 non-null    object
 11  italian      277 non-null    object
 12  xhosa        84 non-null     object
 13  ukrainian    76 non-null     object
 14  french       312 non-null    object
 15  serbian      24 non-null     object
 16  esperanto    57 non-null     object
 17  japanese     2287 non-null   object
 18  latin        227 non-null    object
 19  shona        311 non-null  

In [ ]:
"""
Normalizzazione delle colonne 'author' e 'artist' nel dataset di MangaDex.
Rimuove caratteri speciali, converte in minuscolo e pulisce spazi multipli,
ma mantiene le lingue non latine per un matching più accurato.
"""

def normalize_names(name):
    if pd.isna(name):
        return ""
    
    name = name.lower()
    name = re.sub(r"[^\w\s\-']", "", name)
    name = re.sub(r"\s+", " ", name).strip()
    
    return name

mangaworld_df["author_normalized"] = mangadex_df["author"].apply(normalize_names)
mangaworld_df["artist_normalized"] = mangadex_df["artist"].apply(normalize_names)

mangaworld_df.head()


title  \
0  +99 Reinforced Wooden Stick   
1                       .traeH   
2                009 Re:Cyborg   
3  1-nen C-gumi Yoshida Kakari   
4                   1/2 Prince   

                                          alt_titles          author  \
0  ['99 wooden stick', '99 ทอนไมพรอมบวก', '99 har...         Hongsil   
1                                            ['قلب']    HANATSU Yaya   
2    ['009 re cyborg', '009 re사이보그', '009 рекиборг']             NaN   
3  ['1 nen c gumi yoshida kakari', '1年c組吉田係', 'fi...  SAKURAI Shiori   
4  ['12 hoàng tử', '12 princas', '12 wangzi', '12...           Yu Wo   

            artist     type     status  year    views  \
0           Jipery   Manhwa    dropped  2022  1385162   
1     HANATSU Yaya  Oneshot  completed  2008    62678   
2              NaN    Manga  completed  2012    44944   
3   SAKURAI Shiori  Oneshot  completed  2011    46507   
4  Choi Hong Chong   Manhua  completed  2006   386591   

              title_normalized  \
0   99 reinforced wooden stick   
1                        traeh   
2                 009 recyborg   
3  1-nen c-gumi yoshida kakari   
4                    12 prince   

                              alt_titles_by_language  ... albanian czech  \
0  {'slovak': [''99 wooden stick''], 'thai': [''9...  ...      NaN   NaN   
1                              {'arabic': [''قلب'']}  ...      NaN   NaN   
2  {'danish': [''009 re cyborg''], 'korean': [''0...  ...      NaN   NaN   
3  {'swahili': [''1 nen c gumi yoshida kakari'', ...  ...      NaN   NaN   
4  {'vietnamese': [''12 hoàng tử''], 'italian': [...  ...      NaN   NaN   

  kazakh icelandic latvian azerbaijani urdu matched_title_normalized  \
0    NaN       NaN     NaN         NaN  NaN                     <NA>   
1    NaN       NaN     NaN         NaN  NaN                     <NA>   
2    NaN       NaN     NaN         NaN  NaN                     <NA>   
3    NaN       NaN     NaN         NaN  NaN                     <NA>   
4    NaN       NaN     NaN         NaN  NaN                     <NA>   

  author_normalized artist_normalized  
0           hongsil            jipery  
1      hanatsu yaya      hanatsu yaya  
2                                      
3    sakurai shiori    sakurai shiori  
4             yu wo   choi hong chong  

[5 rows x 87 columns]

In [ ]:
mangaworld_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2906 entries, 0 to 2905
Data columns (total 87 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   title                     2906 non-null   object
 1   alt_titles                2855 non-null   object
 2   author                    2609 non-null   object
 3   artist                    2609 non-null   object
 4   type                      2906 non-null   object
 5   status                    2906 non-null   object
 6   year                      2906 non-null   int64 
 7   views                     2906 non-null   int64 
 8   title_normalized          2906 non-null   object
 9   alt_titles_by_language    2906 non-null   object
 10  slovak                    19 non-null     object
 11  thai                      470 non-null    object
 12  english                   1518 non-null   object
 13  chinese                   1653 non-null   object
 14  korean                  

In [ ]:
mangaworld_df.to_csv("mangaworld_optimized.csv", index=False)